# Budget Correlations
to generate the plots below, we utilize the library deepcave. Unfortunately it is incompatible with our processing environment, therefore a new environment has to be created, and used to exectue this code as explained in [the DeepCave documentation](https://automl.github.io/DeepCAVE/main/installation.html)

In [115]:
from deepcave.runs.converters.smac3v2 import SMAC3v2Run
from pathlib import Path
from collections import defaultdict
from deepcave.runs import Status
import pandas as pd
from scipy import stats

In [116]:
def get_budget_correlations(run: SMAC3v2Run, objective_id: int):
    budget_ids = run.get_budget_ids(include_combined=False)

    # Add symmetric correlations; table ready
    correlations_symmetric = defaultdict(dict)

    correlations = defaultdict(dict)
    for budget1_id in budget_ids:
        budget1 = run.get_budget(budget1_id)
        budget1_readable = run.get_budget(budget1_id, human=True)

        for budget2_id in budget_ids:
            budget2 = run.get_budget(budget2_id)
            budget2_readable = run.get_budget(budget2_id, human=True)

            config_ids1 = run.get_configs(budget1, statuses=[Status.SUCCESS]).keys()
            config_ids2 = run.get_configs(budget2, statuses=[Status.SUCCESS]).keys()

            # Combine config ids
            # So it is guaranteed that there is the same number of configs for each budget
            config_ids = set(config_ids1) & set(config_ids2)

            c1, c2 = [], []
            for config_id in config_ids:
                c1 += [
                    run.get_avg_costs(config_id, budget1, statuses=[Status.SUCCESS])[0][
                        objective_id
                    ]
                ]
                c2 += [
                    run.get_avg_costs(config_id, budget2, statuses=[Status.SUCCESS])[0][
                        objective_id
                    ]
                ]

            correlation = round(stats.spearmanr(c1, c2).correlation, 2)
            correlations_symmetric["Budget"][budget2_readable] = budget2_readable  # type: ignore # noqa: E501
            correlations_symmetric[budget1_readable][budget2_readable] = correlation  # type: ignore # noqa: E501

            # Exclude if budget2 is higher than budget1
            if budget2 > budget1:
                continue

            correlations[budget1_readable][budget2_readable] = correlation  # type: ignore
    return correlations, correlations_symmetric

In [117]:
example_run = SMAC3v2Run.from_path(Path("smac3_output/fehring_growing_nn_new_seeded/net2deeper_budget200_final/5/0"))

In [118]:
get_budget_correlations(example_run, 0)

(defaultdict(dict, {1.0: {1.0: 1.0}, 2.0: {1.0: 0.94, 2.0: 1.0}}),
 defaultdict(dict,
             {'Budget': {1.0: 1.0, 2.0: 2.0},
              1.0: {1.0: 1.0, 2.0: 0.94},
              2.0: {1.0: 0.94, 2.0: 1.0}}))

In [119]:
experiments ={
    1: "Random; 4 layers",
    4: "Random; 2 layers",
    6: "Monster; 4 layers",
    5: "Monster; 2 layers"
}
dataframes: dict = {}

In [120]:
for run_id, experiment in experiments.items():
    run = SMAC3v2Run.from_path(Path(f"smac3_output/fehring_growing_nn_new_seeded/net2deeper_budget200_final/{run_id}/0"))
    correlations, correlations_symmetric = get_budget_correlations(run, 0)
    del correlations_symmetric["Budget"]
    dataframes[experiment] = pd.DataFrame(correlations_symmetric)

In [121]:
dataframes["Random; 2 layers"]


,1.0,2.0
1.0,1.00,0.72
2.0,0.72,1.00


In [124]:
for experiment, df in dataframes.items():
    df.index.name = "Budget"
    df.columns.name = "Budget"
    if len(df) == 4:
        df.index = ["Budget 1", "Budget 2", "Budget 3", "Budget 4"]
        df.columns = ["Budget 1", "Budget 2", "Budget 3", "Budget 4"]
    if len(df) == 2:
        df.index = ["Budget 1", "Budget 2"]
        df.columns = ["Budget 1", "Budget 2"]
    latex_table = df.to_latex(index=True, header=True, float_format="%.2f", caption=f"Correlation Matrix for {experiment}")
    print(experiment)
    print(latex_table)

Random; 4 layers
\begin{table}
\caption{Correlation Matrix for Random; 4 layers}
\begin{tabular}{lrrrr}
\toprule
 & Budget 1 & Budget 2 & Budget 3 & Budget 4 \\
\midrule
Budget 1 & 1.00 & 0.67 & 0.67 & 0.58 \\
Budget 2 & 0.67 & 1.00 & 0.91 & 0.75 \\
Budget 3 & 0.67 & 0.91 & 1.00 & 0.89 \\
Budget 4 & 0.58 & 0.75 & 0.89 & 1.00 \\
\bottomrule
\end{tabular}
\end{table}

Random; 2 layers
\begin{table}
\caption{Correlation Matrix for Random; 2 layers}
\begin{tabular}{lrr}
\toprule
 & Budget 1 & Budget 2 \\
\midrule
Budget 1 & 1.00 & 0.72 \\
Budget 2 & 0.72 & 1.00 \\
\bottomrule
\end{tabular}
\end{table}

Monster; 4 layers
\begin{table}
\caption{Correlation Matrix for Monster; 4 layers}
\begin{tabular}{lrrrr}
\toprule
 & Budget 1 & Budget 2 & Budget 3 & Budget 4 \\
\midrule
Budget 1 & 1.00 & 0.70 & 0.63 & 0.72 \\
Budget 2 & 0.70 & 1.00 & 0.90 & 0.85 \\
Budget 3 & 0.63 & 0.90 & 1.00 & 0.91 \\
Budget 4 & 0.72 & 0.85 & 0.91 & 1.00 \\
\bottomrule
\end{tabular}
\end{table}

Monster; 2 layers
\begi